In [1]:
%matplotlib qt5
# %load_ext cudf.pandas
%load_ext autoreload
%autoreload 2

import mplcursors
import mplfinance as mpf
from kucoin.client import Market
import pandas as pd
import os
from data_class import kucoin_data, kucoin_api, kucoin_fetch_promising, telegramator
import json
import requests
from datetime import datetime, timedelta
import dateparser
from typing import Optional
from binance.exceptions import UnknownDateFormat
import pytz
import ast
from functools import reduce
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import copy 
import re
logging.getLogger().setLevel(logging.INFO)
logging.basicConfig(filename=f'app.log', filemode='a', format='%(asctime)s >>>> %(process)d - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

# import talib

current_directory = os.getcwd()
print(f"Current Directory: {current_directory}")

Current Directory: x:\My Data\Projects\binancebot\2_Kucoin\crypto_finder


In [2]:
keys = json.load(open(f"{current_directory}/keys.json","rb"))
api_type = 'live'
kucoin_rest_api = kucoin_api(api_type, keys[api_type])

In [3]:
how_much_ago = '2 years ago'
default_configs = {            
            # "1hour":{
            #     "interval": "1hour",
            #     "startTime": how_much_ago,
            #     "endTime": "now",
            #     "df_kline": pd.DataFrame(),

            #     "get_rsi": True,
            #     "df_rsi": pd.DataFrame(),

            #     "get_score": False,
            #     "df_score": pd.DataFrame(),

            #     "get_volume": True,
            #     "df_volume": pd.DataFrame(),

            #     "get_stochastic": False,
            #     "df_stochs": pd.DataFrame(),

            #     "get_bollinger": False,
            #     "df_bollinger": pd.DataFrame(),

            #     "get_macd": True,
            #     "df_macd": pd.DataFrame()
            # },
            "1day":{
                "interval": "1day",
                "startTime": how_much_ago,
                "endTime": "now",
                "df_kline": pd.DataFrame(),

                "get_rsi": True,
                "df_rsi": pd.DataFrame(),

                "get_score": False,
                "df_score": pd.DataFrame(),

                "get_volume": True,
                "df_volume": pd.DataFrame(),

                "get_stochastic": False,
                "df_stochs": pd.DataFrame(),

                "get_bollinger": False,
                "df_bollinger": pd.DataFrame(),

                "get_macd": True,
                "df_macd": pd.DataFrame()
            },
            "1week":{
                "interval": "1week",
                "startTime": how_much_ago,
                "endTime": "now",
                "df_kline": pd.DataFrame(),

                "get_rsi": True,
                "df_rsi": pd.DataFrame(),

                "get_score": False,
                "df_score": pd.DataFrame(),

                "get_volume": True,
                "df_volume": pd.DataFrame(),

                "get_stochastic": False,
                "df_stochs": pd.DataFrame(),

                "get_bollinger": False,
                "df_bollinger": pd.DataFrame(),

                "get_macd": True,
                "df_macd": pd.DataFrame()
            }
            }

first_interval = default_configs[list(default_configs.keys())[0]]

## Functions

In [4]:
#region Indicators
bullish_trends = ["Bullish", "Hammer", "Inverted Hammer", "Dragonfly Doji", "Bullish Spinning Top", "Bullish Kicker", "Bullish Engulfing", 
                  "Bullish Harami", "Piercing Line", "Tweezer Bottom", "Morning Star", "Morning Doji Star", "Bullish Abandoned Baby",
                  "Three White Soldiers", "Three Line Strike (Bullish)", "Three Inside Up", "Three Outside Up"]
bearish_trends = ["Bearish", "Hanging Man", "Shooting Star", "Gravestone Doji", "Bearish Spinning Top", "Bearish Kicker", "Bearish Engulfing", "Bearish Harami",
                  "Dark Cloud Cover", "Tweezer Top", "Evening Star", "Evening Doji Star", "Bearish Abandoned Baby", "Three Black Crows", "Three Line Strike (Bearish)",
                  "Three Inside Down", "Three Outside Down"]
neutral = ["Doji"]
#endregion

#region Functions
def prepare_url(url, parameters, api_url):

    url_appendix = ""
    for k in parameters:
        url_appendix+=f"{k}={parameters[k]}&"
    url_appendix = url_appendix[:-1]
    
    return api_url+url+f"?{url_appendix}"

def convert_ts_str(ts_str):
    if ts_str is None:
        return ts_str
    if type(ts_str) == int:
        return ts_str
    return date_to_seconds(ts_str)

def date_to_seconds(date_str: str) -> int:
    """Convert UTC date to milliseconds

    If using offset strings add "UTC" to date string e.g. "now UTC", "11 hours ago UTC"

    See dateparse docs for formats http://dateparser.readthedocs.io/en/latest/

    :param date_str: date in readable format, i.e. "January 01, 2018", "11 hours ago UTC", "now UTC"
    """
    # get epoch value in UTC
    epoch: datetime = datetime.utcfromtimestamp(0).replace(tzinfo=pytz.utc)
    # parse our date string
    d: Optional[datetime] = dateparser.parse(date_str, settings={'TIMEZONE': "UTC"})
    if not d:
        raise UnknownDateFormat(date_str)

    # if the date is not timezone aware apply UTC timezone
    if d.tzinfo is None or d.tzinfo.utcoffset(d) is None:
        d = d.replace(tzinfo=pytz.utc)

    # return the difference in time
    return int((d - epoch).total_seconds())
#endregion

## Fetch Data

In [20]:
%load_ext autoreload
%autoreload 2

symbol = "SILLY-USDT"
local_thread = kucoin_data(
    app=None, 
    kucoin_api_obj=kucoin_rest_api, 
    symbol=symbol, 
    base_asset=symbol.split("-")[0], 
    quote_asset=symbol.split("-")[1],
    default_configs=copy.deepcopy(default_configs), 
    delay=1,
    timeframe_result='D'
    )

local_thread.max_duration_days = 730
local_thread.get_multiple_klines_data()
local_thread.generate_indicator()

if local_thread.combined_indicators.empty:
    raise Exception(f"Data unavailable for {symbol}")

df_indicator = local_thread.combined_indicators
df_indicator = df_indicator.sort_values("Open time", ascending=True)
cols_duplicates = [i for i in df_indicator.columns if i.split("_")[0] in ["Open", "Close", "High", "Low"]]

df_indicator = df_indicator.drop_duplicates(subset=cols_duplicates)
# df_indicator = df_indicator[len(df_indicator)//2:]

df_candlestick = df_indicator.copy()
df_candlestick['Open time'] = pd.to_datetime(df_candlestick['Open time'])

interval = '1day'
df_candlestick = df_candlestick.set_index('Open time')[[f'Open_{interval}', f'High_{interval}', f'Low_{interval}', f'Close_{interval}']]
df_candlestick.rename(columns={f'Open_{interval}':'Open', f'Close_{interval}':'Close', f'High_{interval}':'High', f'Low_{interval}':'Low'}, inplace=True)

print(df_indicator.shape)
display(df_indicator.head(2))
display(df_indicator.tail(2))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(343, 45)


x:\My Data\Projects\binancebot\2_Kucoin\crypto_finder\data_class.py:1258: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  self.combined_indicators = self.combined_indicators.interpolate(method='linear', limit_area='inside')
x:\My Data\Projects\binancebot\2_Kucoin\crypto_finder\data_class.py:1286: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.combined_indicators = self.combined_indicators.ffill().bfill()


,Open time,Open_1day,Close_1day,High_1day,Low_1day,candle_diff_1day,Volume_1day,rsi_1day,stochastic_rsi_k_1day,trend_rsi_1day,...,stochrsi_direction_1week,macd_1week,signal_line_1week,macd_histogram_pct_1week,trend_macd_1week,slope_macd_1week,trend_signal_line_1week,slope_signal_line_1week,trend_macd_histogram_pct_1week,slope_macd_histogram_pct_1week
0,2023-12-27 00:00:00,0.09144,0.14692,0.17601,0.09144,37.762047,8.500735e+06,0.0,100.0,Insufficient data,...,True,-0.005456,-0.001091,-400.0,Downtrend,0.005456,Downtrend,0.001091,No clear trend,0.0
1,2023-12-28 00:00:00,0.14694,0.12104,0.15162,0.11372,-21.397885,7.240280e+06,0.0,100.0,No clear trend,...,True,-0.005456,-0.001091,-400.0,Downtrend,0.005456,Downtrend,0.001091,No clear trend,0.0


,Open time,Open_1day,Close_1day,High_1day,Low_1day,candle_diff_1day,Volume_1day,rsi_1day,stochastic_rsi_k_1day,trend_rsi_1day,...,stochrsi_direction_1week,macd_1week,signal_line_1week,macd_histogram_pct_1week,trend_macd_1week,slope_macd_1week,trend_signal_line_1week,slope_signal_line_1week,trend_macd_histogram_pct_1week,slope_macd_histogram_pct_1week
341,2024-12-02 00:00:00,0.01553,0.01501,0.01622,0.01369,-3.464357,1.918066e+07,56.188885,90.566224,Downtrend,...,True,-0.004749,-0.007058,32.713938,Uptrend,-0.000688,Uptrend,-0.000576,Uptrend,-4.007973
342,2024-12-03 00:00:00,0.01505,0.01518,0.01579,0.01380,0.856390,1.097351e+07,56.863520,92.300464,Uptrend,...,True,-0.004749,-0.007058,32.713938,Uptrend,-0.000717,Uptrend,-0.000576,Uptrend,-4.420439


In [21]:
# BEFORE BULL
data = df_indicator.copy()

timeframe = '1week'
if timeframe in list(default_configs.keys()):
    data[f'is_macd_histogram_pct_uptrend_{timeframe}'] = data.apply(lambda x:x[f'slope_macd_histogram_pct_{timeframe}'] < 0, axis=1)
    data = data[data[f'is_macd_histogram_pct_uptrend_{timeframe}']==True]

    data[f'is_stochrsi_uptrend_{timeframe}'] = data.apply(lambda x:x[f'slope_stochastic_rsi_k_{timeframe}'] <= 0, axis=1)
    data = data[data[f'is_stochrsi_uptrend_{timeframe}']==True]

    data[f'is_stochrsi_slope_downwards_{timeframe}'] = data[f'slope_stochastic_rsi_k_{timeframe}'].shift(1) >= data[f'slope_stochastic_rsi_k_{timeframe}']
    data = data[data[f'is_stochrsi_slope_downwards_{timeframe}']==True]

    data[f'is_stochrsi_high_{timeframe}'] = data.apply(lambda x:x[f'stochastic_rsi_k_{timeframe}'] > 30, axis=1)
    data = data[data[f'is_stochrsi_high_{timeframe}']==True]

    # data[f'is_stochrsi_not_too_high_{timeframe}'] = data.apply(lambda x:x[f'stochastic_rsi_k_{timeframe}'] < 100, axis=1)
    # data = data[data[f'is_stochrsi_not_too_high_{timeframe}']==True]

timeframe = '1day'
if timeframe in list(default_configs.keys()):
    data[f'is_candle_positive_{timeframe}'] = data.apply(lambda x:x[f'candle_diff_{timeframe}'] > 0, axis=1)
    data = data[data[f'is_candle_positive_{timeframe}']==True]

    data[f'is_stochrsi_uptrend_{timeframe}'] = data.apply(lambda x:x[f'slope_stochastic_rsi_k_{timeframe}'] <= 0, axis=1)
    data = data[data[f'is_stochrsi_uptrend_{timeframe}']==True]

    # data[f'stochrsi_direction_{timeframe}'] = data[f'slope_stochastic_rsi_k_{timeframe}'].shift(1) > data[f'slope_stochastic_rsi_k_{timeframe}']
    # data = data[data[f'stochrsi_direction_{timeframe}']==True]

    data[f'is_stochrsi_high_{timeframe}'] = data.apply(lambda x:x[f'stochastic_rsi_k_{timeframe}'] > 30, axis=1)
    data = data[data[f'is_stochrsi_high_{timeframe}']==True]

    data[f'is_stochrsi_not_too_high_{timeframe}'] = data.apply(lambda x:x[f'stochastic_rsi_k_{timeframe}'] < 100, axis=1)
    data = data[data[f'is_stochrsi_not_too_high_{timeframe}']==True]

timeframe = '1hour'
if timeframe in list(default_configs.keys()):
    data[f'is_stochrsi_uptrend_{timeframe}'] = data.apply(lambda x:x[f'slope_stochastic_rsi_k_{timeframe}'] < 0, axis=1)
    data = data[data[f'is_stochrsi_uptrend_{timeframe}']==True]
    
    data[f'is_stochrsi_high_{timeframe}'] = data.apply(lambda x:x[f'stochastic_rsi_k_{timeframe}'] > 30, axis=1)
    data = data[data[f'is_stochrsi_high_{timeframe}']==True]

    data[f'is_stochrsi_not_too_high_{timeframe}'] = data.apply(lambda x:x[f'stochastic_rsi_k_{timeframe}'] < 100, axis=1)
    data = data[data[f'is_stochrsi_not_too_high_{timeframe}']==True]

before_bull = data.copy()
print(before_bull.shape)

(25, 53)


In [22]:
if False:
    fig, ax = plt.subplots(2, 1, figsize=(16,9), sharex=True)
    fig.suptitle(f"Analysis_{symbol}", fontsize=16)
    axs = ax.flatten()

    interval = '1day'

    plt_ind = 0
    mpf.plot(
        df_candlestick,
        type='candle',
        ax=ax[plt_ind],  # Use the subplot axis
        style='yahoo',  # Style of the candlestick plot
        ylabel=f'{interval} Candlestick Plot',  # Add ylabel,
    )

    interval = '1week'

    plt_ind = 1
    ax[plt_ind].plot(df_indicator['Open time'], df_indicator[f'slope_stochastic_rsi_k_{interval}'], label=f'slope_stochastic_rsi_k_{interval}')
    ax[plt_ind].axhline(linewidth=1, color='k', alpha=0.5)
    ax[plt_ind].legend()
    
    for label in before_bull['Open time']:
        ax[0].axvline(x=label, color='green', linestyle='--', alpha=0.5)
else:
    fig, ax = plt.subplots(7, 1, figsize=(16,9), sharex=True)
    fig.suptitle(f"Analysis_{symbol}", fontsize=16)
    axs = ax.flatten()

    interval = '1day'

    plt_ind = 0
    mpf.plot(
        df_candlestick,
        type='candle',
        ax=ax[plt_ind],  # Use the subplot axis
        style='yahoo',  # Style of the candlestick plot
        ylabel=f'{interval} Candlestick Plot',  # Add ylabel,
    )

    interval = '1week'

    plt_ind = 1
    ax[plt_ind].plot(df_indicator['Open time'], df_indicator[f'slope_macd_histogram_pct_{interval}'], label=f'slope_macd_histogram_pct_{interval}')
    ax[plt_ind].axhline(linewidth=1, color='k', alpha=0.5)
    ax[plt_ind].legend()

    plt_ind = 2
    ax[plt_ind].plot(df_indicator['Open time'], df_indicator[f'slope_stochastic_rsi_k_{interval}'], label=f'slope_stochastic_rsi_k_{interval}')
    ax[plt_ind].axhline(linewidth=1, color='k', alpha=0.5)
    ax[plt_ind].legend()

    plt_ind = 3
    ax[plt_ind].plot(df_indicator['Open time'], df_indicator[f'stochastic_rsi_k_{interval}'], label=f'stochastic_rsi_k_{interval}')
    ax[plt_ind].axhline(linewidth=1, color='k', alpha=0.5)
    ax[plt_ind].legend()

    interval = '1day'

    plt_ind = 4
    ax[plt_ind].plot(df_indicator['Open time'], df_indicator[f'slope_macd_histogram_pct_{interval}'], label=f'slope_macd_histogram_pct_{interval}')
    ax[plt_ind].axhline(linewidth=1, color='k', alpha=0.5)
    ax[plt_ind].legend()

    plt_ind = 5
    ax[plt_ind].plot(df_indicator['Open time'], df_indicator[f'slope_stochastic_rsi_k_{interval}'], label=f'slope_stochastic_rsi_k_{interval}')
    ax[plt_ind].axhline(linewidth=1, color='k', alpha=0.5)
    ax[plt_ind].legend()

    plt_ind = 6
    ax[plt_ind].plot(df_indicator['Open time'], df_indicator[f'stochastic_rsi_k_{interval}'], label=f'stochastic_rsi_k_{interval}')
    ax[plt_ind].axhline(linewidth=1, color='k', alpha=0.5)
    ax[plt_ind].legend()

    # plt_ind = 4
    # ax[plt_ind].plot(df_indicator['Open time'], df_indicator[f'macd_{interval}'], label=f'Open_{interval}', color='blue')
    # ax[plt_ind].plot(df_indicator['Open time'], df_indicator[f'signal_line_{interval}'], label=f'signal_line_{interval}', color='orange')
    # ax[plt_ind].axhline(linewidth=1, color='k', alpha=0.5)
    # ax[plt_ind].legend()

    mplcursors.cursor(ax[1].lines + ax[2].lines + ax[3].lines + ax[4].lines + ax[5].lines + ax[6].lines, hover=False)

    for label in before_bull['Open time']:
        ax[0].axvline(x=label, color='green', linestyle='--', alpha=0.5)

plt.xticks(rotation=45, fontsize=7) 
pass